In [ ]:
from gensim import corpora, models, similarities
import json
from nltk.tokenize import word_tokenize
from gensim.similarities import MatrixSimilarity

Importando arquivos com o map

In [ ]:
try:
   with open('map_docs.json', 'r', encoding='utf-8') as file:
       map_docs = json.load(file)
   with open('map_queries.json', 'r', encoding='utf-8') as file:
       map_queries = json.load(file)
except FileNotFoundError as e:
    print(f"Erro ao abrir o arquivo: {e}")

Criando modelo TF-IDF

In [ ]:
# Listas paralelas para manter mapeamento entre índice e doc_id
doc_ids = []
texts = []

for doc_id, texto in map_docs.items():
    tokens = word_tokenize(texto.lower())
    texts.append(tokens)
    doc_ids.append(doc_id)  # manter a ordem original

In [ ]:
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
tfidf_model = models.TfidfModel(bow_corpus)

In [ ]:
from gensim.similarities import Similarity

# Crie o índice de similaridade eficiente para grandes corpora
index = Similarity(output_prefix='tfidf_index', corpus=tfidf_model[bow_corpus], num_features=len(dictionary))

In [ ]:
def buscar_tfidf(query, top_n=100):
    query_tokens = word_tokenize(query.lower())
    query_bow = dictionary.doc2bow(query_tokens)
    query_tfidf = tfidf_model[query_bow]
    sims = index[query_tfidf]
    ranked_indices = sims.argsort()[::-1][:top_n]
    # Retorna apenas os doc_ids, sem score
    return [doc_ids[i] for i in ranked_indices]

Consultas

In [ ]:
# Para todas as consultas em map_queries, obter os documentos e armazenar os resultados em map_results (apenas o doc id, sem score)
map_results = {}
for query_id, query_text in map_queries.items():
    resultados = buscar_tfidf(query_text)
    map_results[query_id] = resultados

In [ ]:
map_results

In [ ]:
# Salvar map_results em um arquivo JSON map_results_mysql.json
with open('map_results_tfidf.json', 'w', encoding='utf-8') as file:
    json.dump(map_results, file, ensure_ascii=False, indent=4)